In [28]:
import cv2
import pandas as pd
import numpy as np
from os import walk
from matplotlib import pyplot as plt
import imutils

# Preprocessing screenshots from 3D CAD models

In [31]:
CAD_SCREENSHOTS = "/Users/mariusoechslein/Documents/MyDocuments/Studium/Master/3d-parts-project/3d_mai/data/val/"

In [38]:
import os

def calculate_cropping_indices(img_shape):
    real_aspect_ratio = img_shape[0] / img_shape[1]
    goal_x = (9/16) * img_shape[1]
    x_mid = img.shape[0] / 2
    new_x_start = int(x_mid - (goal_x/2))
    new_x_end = int(x_mid + (goal_x/2))
    return new_x_start, new_x_end

def transpose_bounding_box(path):
    label_file = open(path, "r")
    label_line = label_file.read().split(" ")
    label_file.close()
    label_file = open(path, "w")
    label_line = "".join(str(x) + " " for x in [label_line[0], label_line[2], label_line[1], label_line[4], label_line[3]])
    label_file.write(label_line)

def preprocess_cad_images(CAD_SCREENSHOTS, TARGET_RESOLUTION=(400,225)): 
    IMAGES_FOLDER = CAD_SCREENSHOTS + "images"
    LABELS_FOLDER = CAD_SCREENSHOTS + "labels"
    subdirs = [x[0] for x in os.walk(IMAGES_FOLDER)]
    for subdir in subdirs:
        files = os.walk(subdir).__next__()[2]
        if (len(files) > 0):
            for file in files:
                file_name = file.split(".")[0]
                image_name = os.path.join(subdir, file)
                if image_name[-4:] != ".jpg":
                    continue
            
                image = cv2.imread(image_name)
            
                # Detect the contours
                gray = cv2.bilateralFilter(image, 3, 5, 5)
                edged = cv2.Canny(gray, 100, 200)
                cnts = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                cnts = imutils.grab_contours(cnts)
                img = cv2.cvtColor(edged, cv2.COLOR_GRAY2BGR)  #add this line
                img = cv2.drawContours(img, cnts, -1, 255, 3)
                #im_bw = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]
                img = img[:,:,0]
                
                # Maybe Transpose the images since the IPhone doesn't have 1920x1080 resolutions
                if img.shape[0] > img.shape[1]:
                    img = img.T
                    #transpose_bounding_box(LABELS_FOLDER + file_name + ".txt")

                # Calculate how much to crop to get from IPhone resolution to 16:9 aspect ratio
                #if img.shape[0] / img.shape[1] != 0.5625:
                    #new_x_start, new_x_end = calculate_cropping_indices(img.shape)
                    #img = img[new_x_start:new_x_end, :]
            
                # resize the image to a fixed size. Resize AFTER detecting the edges
                image_resized = cv2.resize(img, TARGET_RESOLUTION) # 800 x 450 = ratio of 1920 x 1080 downscaled
            
                SAVING_PATH = CAD_SCREENSHOTS + "preprocessed/" + file
                # Save preprocessed real image to folder
                cv2.imwrite(SAVING_PATH, image_resized)

print("Only .jpg accepted!")
preprocess_cad_images(CAD_SCREENSHOTS)


# Preprocessing real images

In [21]:
FOLDER_REAL_IMAGES = "/Users/mariusoechslein/Documents/MyDocuments/Studium/Master/3d-parts-project/3d_mai/data/test/images_raw/"
FOLDER_PREPROCESSED_REAL_IMAGES = "/Users/mariusoechslein/Documents/MyDocuments/Studium/Master/3d-parts-project/3d_mai/data/test/images_preprocessed_contrast/"

In [24]:
import os

def calculate_cropping_indices(img_shape):
    real_aspect_ratio = img_shape[0] / img_shape[1]
    goal_x = (9/16) * img_shape[1]
    x_mid = img_shape[0] / 2
    new_x_start = int(x_mid - (goal_x/2))
    new_x_end = int(x_mid + (goal_x/2))
    return new_x_start, new_x_end

def preprocess_real_images(INPUT_FOLDER_PATH, OUTPUT_FOLDER_PATH, TARGET_RESOLUTION=(400,225)): 
    IMAGES_FOLDER = INPUT_FOLDER_PATH 
    subdirs = [x[0] for x in os.walk(IMAGES_FOLDER)]
    for subdir in subdirs:
        files = os.walk(subdir).__next__()[2]
        if (len(files) > 0):
            for file in files:
                file_name = file.split(".")[0]
                image_name = os.path.join(subdir, file)
            
                image = cv2.imread(image_name)

                alpha = 1.5 # Contrast control (1.0-3.0)
                beta = 30 # Brightness control (0-100)
                image = cv2.convertScaleAbs(image, alpha=alpha, beta=beta)
            
                # Detect the contours
                gray = cv2.bilateralFilter(image, 3, 5, 5)
                edged = cv2.Canny(gray, 100, 200)
                cnts = cv2.findContours(edged.copy(), cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                cnts = imutils.grab_contours(cnts)
                img = cv2.cvtColor(edged, cv2.COLOR_GRAY2BGR)  #add this line
                img = cv2.drawContours(img, cnts, -1, 255, 3)
                #im_bw = cv2.threshold(img, 127, 255, cv2.THRESH_BINARY)[1]
                img = img[:,:,0]
            
                # Maybe Transpose the images since the IPhone doesn't have 1920x1080 resolutions
                if img.shape[0] > img.shape[1]:
                    img = img.T
                # Calculate how much to crop to get from IPhone resolution to 16:9 aspect ratio
                if img.shape[0] / img.shape[1] != 0.5625:
                    new_x_start, new_x_end = calculate_cropping_indices(img.shape)
                    img = img[new_x_start:new_x_end, :]
            
                # resize the image to a fixed size. Resize AFTER detecting the edges
                image_resized = cv2.resize(img, TARGET_RESOLUTION) # 800 x 450 = ratio of 1920 x 1080 downscaled
            
                SAVING_PATH = OUTPUT_FOLDER_PATH + "/" + file
                # Save preprocessed real image to folder

                cv2.imwrite(SAVING_PATH, image_resized)

preprocess_real_images(FOLDER_REAL_IMAGES, FOLDER_PREPROCESSED_REAL_IMAGES, TARGET_RESOLUTION=(400,225))